In [4]:
# Desinstalar el SDK viejo


# Instalar el SDK moderno y yfinance
%pip install alpaca-py yfinance

# Instalar dependencias adicionales
%pip install numpy pandas scipy matplotlib requests statsmodels

%pip install alpaca-py --quiet

In [5]:
import os
import datetime
import pandas as pd
import yfinance as yf
import warnings
import numpy as np
from scipy.signal import find_peaks
from scipy.stats import linregress
from statsmodels.tsa.arima.model import ARIMA

# --- IMPORTACIONES DE ALPACA ---
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

warnings.filterwarnings("ignore")

# ==============================================================================
# 1. CONFIGURACIÓN
# ==============================================================================
API_KEY = "PKWHQAEKVEX2Q6UEJPSWR6EPK7"
API_SECRET = "6rHRpN2REy8cH9r5C6vWwiExr2yXXsm5XwqvKteMNVAx"

TICKERS = ["AAPL", "META", "AMZN", "MSFT", "GOOGL", "TSLA", "NVDA", "GE", "HD", "CAT", "WMT"]
TIMEFRAME = "1Day"
PAPER_TRADING = True

if API_KEY is None:
    print("⚠️ ERROR CRÍTICO: No se encontraron las claves API.")
else:
    data_client = StockHistoricalDataClient(API_KEY, API_SECRET)
    trading_client = TradingClient(API_KEY, API_SECRET, paper=PAPER_TRADING)

SESSION_LOG = []
START_EQUITY = 0.0

# ==============================================================================
# 2. MOTORES DE DATOS
# ==============================================================================
def get_data(ticker):
    end_dt = datetime.datetime.now()
    start_dt = end_dt - datetime.timedelta(days=400)
    try:
        tf = TimeFrame.Day if TIMEFRAME == "1Day" else TimeFrame.Hour
        req = StockBarsRequest(symbol_or_symbols=ticker, timeframe=tf, start=start_dt.date(), end=end_dt.date())
        bars = data_client.get_stock_bars(req)
        if not bars or ticker not in bars.data: raise ValueError("Datos vacíos")
        df = bars.df.reset_index()
        df = df[df['symbol'] == ticker].set_index('timestamp')
        return df.rename(columns={"close": "Close", "high": "High", "low": "Low", "open": "Open", "volume": "Volume"})
    except:
        df = yf.download(ticker, start=start_dt, end=end_dt, progress=False)
        return df.rename(columns={"Adj Close": "Close"})

def get_weekly_options_chain(ticker, current_price):
    try:
        stock = yf.Ticker(ticker)
        if not stock.options: return pd.DataFrame(), pd.DataFrame(), "N/A"

        target_date = datetime.date.today() + datetime.timedelta(days=7)
        exp_date = min(stock.options, key=lambda x: abs(pd.to_datetime(x).date() - target_date))

        chain = stock.option_chain(exp_date)
        calls = chain.calls
        puts = chain.puts

        calls = calls[(calls['strike'] >= 0.8 * current_price) & (calls['strike'] <= 1.2 * current_price)]
        puts = puts[(puts['strike'] >= 0.8 * current_price) & (puts['strike'] <= 1.2 * current_price)]
        return calls, puts, exp_date
    except: return pd.DataFrame(), pd.DataFrame(), "N/A"

# ==============================================================================
# 3. PRONÓSTICOS (ARIMA)
# ==============================================================================
def get_arima_forecast(series, steps=5):
    try:
        history = series.iloc[-100:].values.flatten()
        model = ARIMA(history, order=(5, 1, 0))
        model_fit = model.fit()
        forecast = model_fit.forecast(steps=steps)

        target_price = forecast[-1]
        pct_change = ((target_price - history[-1]) / history[-1]) * 100

        direction = "SUBE" if pct_change > 0 else "BAJA"
        return direction, target_price, pct_change
    except: return "ERROR", 0, 0

# ==============================================================================
# 4. PATRONES E INDICADORES
# ==============================================================================
def get_picos_valles(series, order_n=5):
    vals = series.values.flatten()
    picos_idx = find_peaks(vals, distance=order_n)[0]
    valles_idx = find_peaks(-vals, distance=order_n)[0]
    return picos_idx, vals[picos_idx], valles_idx, vals[valles_idx]

def get_trendline(indices, values):
    if len(indices) < 2: return None
    res = linregress(indices.flatten(), values.flatten())
    return res.slope, res.intercept

def is_near(val1, val2, tol=0.03):
    if val2 == 0: return abs(val1) <= tol
    return abs(val1 - val2) / abs(val2) <= tol

def detect_advanced_patterns(df, current_price):
    if df.empty: return "NEUTRAL"
    close = df['Close']
    p_idx, p_val, v_idx, v_val = get_picos_valles(close)

    if len(p_idx) >= 3:
        P1, P2, P3 = p_val[-3:]
        if P2 > P1 and P2 > P3 and is_near(P1, P3, 0.05):
             if current_price < min(v_val[-2:]): return "HCH_BAJISTA"
    if len(v_idx) >= 3:
        V1, V2, V3 = v_val[-3:]
        if V2 < V1 and V2 < V3 and is_near(V1, V3, 0.05):
            if current_price > max(p_val[-2:]): return "HCH_INV_ALCISTA"
    if len(p_idx) >= 3 and len(v_idx) >= 3:
        res_slope, _ = get_trendline(p_idx[-3:], p_val[-3:])
        sup_slope, _ = get_trendline(v_idx[-3:], v_val[-3:])
        if abs(res_slope) < 0.05 and sup_slope > 0.05:
            if current_price > p_val[-1]: return "TRIANGULO_ASC_ALCISTA"
        if abs(sup_slope) < 0.05 and res_slope < -0.05:
            if current_price < v_val[-1]: return "TRIANGULO_DESC_BAJISTA"
    if len(v_idx) >= 2 and is_near(v_val[-1], v_val[-2]) and current_price > p_val[-1]: return "DOBLE_SUELO_ALCISTA"
    if len(p_idx) >= 2 and is_near(p_val[-1], p_val[-2]) and current_price < v_val[-1]: return "DOBLE_TECHO_BAJISTA"
    return "NEUTRAL"

def calculate_indicators(df):
    close = df['Close']
    sma_50 = close.rolling(window=50).mean()
    sma_200 = close.rolling(window=200).mean()

    # Bollinger Bands (Para riesgo)
    sma_20 = close.rolling(window=20).mean()
    std_dev = close.rolling(window=20).std()
    bb_upper = sma_20 + (std_dev * 2)
    bb_lower = sma_20 - (std_dev * 2)

    # RSI & MACD
    delta = close.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    rs = gain.ewm(alpha=1/14).mean() / loss.ewm(alpha=1/14).mean()
    rsi = 100 - (100 / (1 + rs))

    ema12 = close.ewm(span=12).mean()
    ema26 = close.ewm(span=26).mean()
    macd = ema12 - ema26
    signal = macd.ewm(span=9).mean()

    return {
        'sma_50': sma_50.iloc[-1], 'sma_200': sma_200.iloc[-1],
        'bb_upper': bb_upper.iloc[-1], 'bb_lower': bb_lower.iloc[-1],
        'rsi': rsi.iloc[-1], 'macd': macd.iloc[-1], 'macd_sig': signal.iloc[-1]
    }

# ==============================================================================
# 5. LÓGICA DE DECISIÓN
# ==============================================================================
def get_signal(ticker):
    try:
        df = get_data(ticker)
        if len(df) < 200: return "HOLD", 0, 0, 0, "Datos Insuf.", "N/A", "N/A"

        price = df['Close'].iloc[-1]
        stats = calculate_indicators(df)
        pattern = detect_advanced_patterns(df, price)

        # Pronósticos
        arima_dir, arima_target, arima_pct = get_arima_forecast(df['Close'])
        forecast_str = f"{arima_dir} {arima_pct:.1f}%"

        # Filtro Tendencia
        trend_ma = stats['sma_200'] if not np.isnan(stats['sma_200']) else stats['sma_50']
        is_uptrend = price > trend_ma

        buy_signal = False
        sell_signal = False

        # Señales
        if "ALCISTA" in pattern and is_uptrend: buy_signal = True
        if "BAJISTA" in pattern: sell_signal = True

        if pattern == "NEUTRAL":
            if is_uptrend and stats['macd'] > stats['macd_sig'] and stats['rsi'] < 70:
                buy_signal = True
            if not is_uptrend or stats['rsi'] > 75:
                sell_signal = True

        if buy_signal and arima_pct < -2.0:
            buy_signal = False

        # --- OPCIONES & IMPRESIÓN (MODIFICADO) ---
        calls, puts, exp_date = get_weekly_options_chain(ticker, price)

        opt_type = "N/A"
        opt_str = "N/A"
        opt_exp = "N/A"
        option_rec = "Sin Acción"

        if buy_signal and not calls.empty:
            atm = calls.iloc[(calls['strike'] - price).abs().argsort()[:1]]
            opt_type = "CALL"
            opt_str = f"{atm['strike'].values[0]}"
            opt_exp = str(exp_date)
            cost = atm['lastPrice'].values[0]
            option_rec = f"BUY CALL | Exp: {opt_exp} | Str: {opt_str} | ${cost}"

        elif sell_signal and not puts.empty:
            atm = puts.iloc[(puts['strike'] - price).abs().argsort()[:1]]
            opt_type = "PUT"
            opt_str = f"{atm['strike'].values[0]}"
            opt_exp = str(exp_date)
            cost = atm['lastPrice'].values[0]
            option_rec = f"BUY PUT  | Exp: {opt_exp} | Str: {opt_str} | ${cost}"

        # Gestión de Riesgo
        bb_lower = stats['bb_lower']
        bb_upper = stats['bb_upper']
        sl = bb_lower if bb_lower < price else price * 0.98
        tp = bb_upper if bb_upper > price else price * 1.02

        # --- OUTPUT MODIFICADO: TICKER, PRECIO, PATRON, OPCION DETALLES, ARIMA ---
        print(f"    📊 {ticker:<5} | ${price:<7.2f} | Pat: {pattern:<22} | Opt: {opt_type:<4} | Exp: {opt_exp:<10} | Str: {opt_str:<6} | Cast: {arima_dir} {arima_pct:.1f}%")

        if buy_signal: return "BUY", price, sl, tp, pattern, option_rec, forecast_str
        if sell_signal: return "SELL", price, 0, 0, pattern, option_rec, forecast_str
        return "HOLD", price, 0, 0, "NEUTRAL", "Sin Acción", forecast_str

    except Exception as e:
        print(f"    ⚠️ Error {ticker}: {e}")
        return "HOLD", 0, 0, 0, "Error", "Error", "Error"

# ==============================================================================
# 6. EJECUCIÓN
# ==============================================================================
def execute_trade(ticker, signal, price, sl, tp, pattern, opt_rec, forecast):
    try:
        try:
            pos = trading_client.get_open_position(ticker)
            qty_held = float(pos.qty)
        except: qty_held = 0

        if signal == "BUY":
            if qty_held == 0:
                qty = max(1, int(1000 / price))
                req = MarketOrderRequest(
                    symbol=ticker, qty=qty, side=OrderSide.BUY, time_in_force=TimeInForce.DAY,
                    order_class="bracket",
                    take_profit={"limit_price": round(tp, 2)},
                    stop_loss={"stop_price": round(sl, 2)}
                )
                trading_client.submit_order(req)
                print(f"       🚀 ORDEN ENVIADA: Comprar {qty} acciones")
                SESSION_LOG.append({"Ticker": ticker, "Accion": "COMPRA", "Cant": qty, "Precio": price, "Patron": pattern, "Opcion": opt_rec, "Pronostico": forecast})
            else: print(f"       🚫 Skip: Ya tienes {qty_held}")

        elif signal == "SELL":
            if qty_held > 0:
                req = MarketOrderRequest(symbol=ticker, qty=qty_held, side=OrderSide.SELL, time_in_force=TimeInForce.DAY)
                trading_client.submit_order(req)
                print(f"       📉 ORDEN ENVIADA: Vender {qty_held} acciones")
                SESSION_LOG.append({"Ticker": ticker, "Accion": "VENTA", "Cant": qty_held, "Precio": price, "Patron": pattern, "Opcion": opt_rec, "Pronostico": forecast})
            else: print(f"       🚫 Skip: No tienes posición")

    except Exception as e: print(f"       ❌ Error Ejecución: {e}")

# ==============================================================================
# 7. REPORTE
# ==============================================================================
def print_report():
    print("\n" + "="*95)
    print(f"🛑 REPORTE DE SESIÓN | {datetime.datetime.now().strftime('%Y-%m-%d %H:%M')}")
    print("="*95)
    if SESSION_LOG:
        print(f"{'TICKER':<6} {'ACCIÓN':<8} {'CANT':<5} {'PRECIO':<8} {'PATRÓN':<22} {'OPCIÓN RECOMENDADA'} {'PRONÓSTICO'}")
        print("-" * 95)
        for t in SESSION_LOG:
            print(f"{t['Ticker']:<6} {t['Accion']:<8} {t['Cant']:<5} ${t['Precio']:<7.2f} {t['Patron']:<22} {t['Opcion']} {t['Pronostico']}")
    else: print("   No se ejecutaron operaciones.")
    print("="*95 + "\n")

def run_bot():
    print("\n🤖 BOT HÍBRIDO")
    if API_KEY is None: print("❌ Faltan claves API"); return
    try:
        if not trading_client.get_clock().is_open: print("🌑 Mercado CERRADO")
        else: print("☀️ Mercado ABIERTO")
    except: pass

    print(f"\n🔍 Escaneando {len(TICKERS)} acciones...")
    for ticker in TICKERS:
        s, p, sl, tp, pat, opt, fcast = get_signal(ticker)
        if s != "HOLD": execute_trade(ticker, s, p, sl, tp, pat, opt, fcast)
    print_report()
    print("✅ Sesión Finalizada.")

if __name__ == "__main__":
    run_bot()


🤖 BOT HÍBRIDO
🌑 Mercado CERRADO

🔍 Escaneando 11 acciones...
    📊 AAPL  | $266.25  | Pat: HCH_BAJISTA            | Opt: PUT  | Exp: 2025-11-28 | Str: 267.5  | Cast: BAJA -0.2%
       🚫 Skip: No tienes posición
    📊 META  | $589.15  | Pat: NEUTRAL                | Opt: PUT  | Exp: 2025-11-28 | Str: 590.0  | Cast: SUBE 0.1%
       🚫 Skip: No tienes posición
    📊 AMZN  | $217.14  | Pat: NEUTRAL                | Opt: N/A  | Exp: N/A        | Str: N/A    | Cast: BAJA -0.2%
    📊 MSFT  | $478.43  | Pat: HCH_BAJISTA            | Opt: PUT  | Exp: 2025-11-28 | Str: 477.5  | Cast: BAJA -0.0%
       🚫 Skip: No tienes posición
    📊 GOOGL | $289.45  | Pat: NEUTRAL                | Opt: N/A  | Exp: N/A        | Str: N/A    | Cast: SUBE 0.9%
    📊 TSLA  | $395.23  | Pat: NEUTRAL                | Opt: N/A  | Exp: N/A        | Str: N/A    | Cast: BAJA -0.0%
    📊 NVDA  | $180.64  | Pat: NEUTRAL                | Opt: N/A  | Exp: N/A        | Str: N/A    | Cast: SUBE 0.9%
    📊 GE    | $290.62  | Pa